In [1]:
import os 
import glob 
from tqdm import tqdm

import torch
import pandas as pd
import numpy as np
import skimage.metrics 
from torchmetrics.functional.classification import dice, recall, precision
from torchmetrics import Dice, Recall, Precision 
import nibabel as nib 

In [2]:
def binarize_image(img, threshold = 0.5, one_hot = False):
    if img.ndim == 4:
       img = img.unsqueeze(0)

    elif img.ndim == 3:
        img = img[None, None, :, :, :]

    assert img.ndim == 5, f'Binarize_image, tensor mismatch {img.shape}'

    n_channels = img.shape[1]

    # binary problem
    if n_channels == 1:
        nimg = img > threshold
    elif n_channels == 3:
        if img.dtype == torch.bool:
            nimg = img.float()
        else:
            nimg           = torch.zeros_like(img)
            argmax_indexes = torch.argmax(img, dim = 1)
            nimg.scatter_(1, argmax_indexes.unsqueeze(1), 1) 
    else:
        print(f"In binarize_image, number of channels {n_channels}")
    
    if nimg.dtype != torch.float:   nimg = nimg.float()
    
    return nimg

def calculate_overlap_metrics(pred, gt, target_label: int == 1):
    aneur_mask      = torch.where(gt == target_label, 1, 0)
    pred_image_bin  = binarize_image(pred)
    pred_aneur_mask = torch.mul(pred_image_bin, aneur_mask)

    # compute dice score recall and precision
    tp = torch.sum((pred_aneur_mask == 1) & (aneur_mask == 1))
    fp = torch.sum((pred_aneur_mask == 1) & (aneur_mask == 0))
    fn = torch.sum((pred_aneur_mask == 0) & (aneur_mask == 1))

    if 2*tp + fp + fn == 0: dice_aneur = 1e-12
    else: dice_aneur = (2*tp/(2*tp + fp + fn)).item()

    if tp + fn == 0: recall_aneur = 1e-12
    else: recall_aneur = (tp/(tp+fn)).item()

    if fp + tp == 0: precision_aneur = 1e-12
    else: precision_aneur = (tp/(tp+fp)).item()
    metrics = {'dice_aneur':dice_aneur, 
               'recall_aneur':recall_aneur, 
               'precision_aneur':precision_aneur}
    
    return metrics

In [30]:
def calculate_metrics(num_classes, gt_vols_fp, pred_vols_fn,
                      collapse_into_single_uia_class=False, untreated_aneurysm_only=False):
    dice = Dice(num_classes=num_classes, ignore_index=0, average='micro')
    recall = Recall(num_classes=num_classes, num_labels=num_classes, ignore_index=0, average='micro', task='binary')
    precision = Precision(num_classes=num_classes,  num_labels=num_classes, ignore_index=0, average='micro', task='binary')
    
    metrics_tm = {'dice': dice, 'recall': recall, 'precision': precision}
    
    results = []
    for gt_vol_fp in tqdm(gt_vols_fp):
        vol_fn = os.path.basename(gt_vol_fp)
    
        try:
            assert vol_fn in pred_vols_fn, \
                f"No prediction for vol {vol_fn}"
        except AssertionError as e:
            print(e)
            continue
        
        # load vols     
        gt = nib.load(gt_vol_fp).get_fdata()
        gt = torch.tensor(gt).int()
        
        pred = nib.load(os.path.join(predictions_dir, vol_fn)).get_fdata()
        pred = torch.tensor(pred).int()
           
        # Calculate metrics
        metrics = calculate_overlap_metrics(pred.float(), gt.float(), target_label= 1)
        metrics = {f'{k}_kostas':v for k,v in metrics.items()}
        
        for metric_name, metric_tm in metrics_tm.items():
            try:
                metrics[f'{metric_name}_tm'] = metric_tm(pred, gt).item()
            except:
                print(f'Error calculating {metric_name} for {vol_fn}')
                metrics[f'{metric_name}_tm'] = np.nan
                
        metrics['mhd'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')
        
        if collapse_into_single_uia_class and num_classes > 2:
            # collapse both labels into a single aneurysm class
            pred_mod = torch.where(pred > 0, 1, 0)
            gt_mod = torch.where(gt > 0, 1, 0)
            
            for metric_name, metric_tm in metrics_tm.items():
                metrics[f'{metric_name}_tm_single_UIA_class'] = metric_tm(pred_mod, gt_mod).item()
                metrics['mhd_single_UIA_class'] = skimage.metrics.hausdorff_distance(gt_mod.cpu().numpy(), pred_mod.cpu().numpy(), method='modified')
                
        if untreated_aneurysm_only and num_classes > 2:
            # Calculate the aneurysm only metrics
            gt = torch.where(gt == 1, 1, 0)
            pred = torch.where(pred == 1, 1, 0)
            
            for metric_name, metric_tm in metrics_tm.items():
                 metrics[f'{metric_name}_tm_untreated_aneurysm_only'] = metric_tm(pred, gt).item()
            metrics['mhd_untreated_aneurysm_only'] = skimage.metrics.hausdorff_distance(gt.cpu().numpy(), pred.cpu().numpy(), method='modified')

        # Add metrics to the results list
        results.append({
            'vol_name': vol_fn,
            **metrics
        })
    
    return pd.DataFrame(results)
    

# Evaluate trained on source domain and predicting in holdout set of the same domain (ADAM)

Keep Dice-Score, Recall, Precision, and modified Hausdorff Distance for each volume and each class


## 3 Classes: Background, Untreated, Treated Aneurysms

In [35]:
data_dir = '../../../data/'
results_dir = os.path.join(data_dir, 'results')

os.makedirs(results_dir, exist_ok=True)

ground_truth_dir = os.path.join(data_dir, 'ADAM/Dataset005_ADAM3ClassesAneurysmsOnly')
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, 'labelsTs', '*.nii.gz'))
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset005_ADAM3ClassesAneurysmsOnly', 'imageTs')
pred_vols_fn = os.listdir(predictions_dir)
pred_vols_fn[0:5]

['10047B.nii.gz',
 '10048B.nii.gz',
 '10029.nii.gz',
 '10024.nii.gz',
 '10068F.nii.gz']

In [36]:
results_df = calculate_metrics(num_classes=3, gt_vols_fp=gt_vols_fp, pred_vols_fn=pred_vols_fn,
                               collapse_into_single_uia_class=True, untreated_aneurysm_only=True)

  9%|▊         | 2/23 [01:47<17:57, 51.33s/it]

Error calculating recall for 10029.nii.gz
Error calculating precision for 10029.nii.gz


 17%|█▋        | 4/23 [03:03<13:38, 43.09s/it]

Error calculating recall for 10068F.nii.gz
Error calculating precision for 10068F.nii.gz


 22%|██▏       | 5/23 [06:05<27:58, 93.25s/it]

...

 87%|████████▋ | 20/23 [18:27<02:07, 42.52s/it]

Error calculating dice for 10010.nii.gz
Error calculating recall for 10010.nii.gz
Error calculating precision for 10010.nii.gz


 91%|█████████▏| 21/23 [19:02<01:20, 40.16s/it]

Error calculating recall for 10060B.nii.gz
Error calculating precision for 10060B.nii.gz


100%|██████████| 23/23 [22:30<00:00, 58.74s/it]


In [37]:
results_df

,vol_name,dice_aneur_kostas,recall_aneur_kostas,precision_aneur_kostas,dice_tm,recall_tm,precision_tm,mhd,dice_tm_single_UIA_class,mhd_single_UIA_class,recall_tm_single_UIA_class,precision_tm_single_UIA_class,dice_tm_untreated_aneurysm_only,recall_tm_untreated_aneurysm_only,precision_tm_untreated_aneurysm_only,mhd_untreated_aneurysm_only
0,10047B.nii.gz,7.530864e-01,6.039604e-01,1.000000e+00,0.721893,0.603960,1.0,0.560829,0.721893,0.560829,0.603960,1.0,0.721893,0.603960,1.0,0.560829
1,10048B.nii.gz,8.642160e-01,7.608983e-01,1.000000e+00,0.789582,0.760898,1.0,0.273209,0.789582,0.273209,0.760898,1.0,0.789582,0.760898,1.0,0.273209
2,10029.nii.gz,9.812108e-01,9.631147e-01,1.000000e+00,0.662906,NaN,NaN,8.247260,0.662906,8.247260,0.963115,1.0,0.690162,0.963115,1.0,0.534330
3,10024.nii.gz,9.649485e-01,9.322709e-01,1.000000e+00,0.717791,0.932271,1.0,0.462330,0.717791,0.462330,0.932271,1.0,0.717791,0.932271,1.0,0.462330
4,10068F.nii.gz,9.463493e-01,8.981623e-01,1.000000e+00,0.779920,NaN,NaN,1.886223,0.779920,1.886223,0.898162,1.0,0.794715,0.898162,1.0,0.576065
5,10072F.nii.gz,0.000000e+00,0.000000e+00,1.000000e-12,0.000000,NaN,NaN,inf,0.000000,inf,0.000000,0.0,0.000000,0.000000,0.0,inf
6,10037.nii.gz,0.000000e+00,0.000000e+00,1.000000e-12,0.000000,0.000000,0.0,inf,0.000000,inf,0.000000,0.0,0.000000,0.000000,0.0,inf
7,10068B.nii.gz,9.455041e-01,8.966408e-01,1.000000e+00,0.288565,NaN,NaN,10.579254,0.288565,10.579254,0.896641,1.0,0.826190,0.896641,1.0,0.233996
8,10072B.nii.gz,0.000000e+00,0.000000e+00,1.000000e-12,0.000000,NaN,NaN,37.656664,0.018338,37.656664,0.009254,1.0,0.000000,0.000000,0.0,75.865834
9,10047F.nii.gz,6.952381e-01,5.328467e-01,1.000000e+00,0.691943,0.532847,1.0,0.572109,0.691943,0.572109,0.532847,1.0,0.691943,0.532847,1.0,0.572109


In [39]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_adam__td_adam__3classes_treated_and_untreated_UIAs.csv'), index=False)

In [40]:
results_df.dice_tm.mean()

0.42088863514363767

In [47]:
results_df.mhd.map(lambda x: x if x > 68 else 68).mean()



AttributeError: 'DataFrame' object has no attribute 'mhd'

### Binary Segmentation Aneurysm vs Background

In [52]:
ground_truth_dir = os.path.join(data_dir, 'ADAM/Dataset006_ADAMBinaryAneurysmsOnly')
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, 'labelsTs', '*.nii.gz'))
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset006_ADAMBinaryAneurysmsOnly', 'imageTs')
pred_vols_fn = os.listdir(predictions_dir)
pred_vols_fn[0:5]

['10047B.nii.gz',
 '10048B.nii.gz',
 '10029.nii.gz',
 '10024.nii.gz',
 '10068F.nii.gz']

In [53]:
gt_vols_fp

['../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10047B.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10048B.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10029.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10024.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10068F.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10072F.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10037.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10068B.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10072B.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10047F.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10015.nii.gz',
 '../../../data/ADAM/Dataset006_ADAMBinaryAneurysmsOnly/labelsTs/10049F.nii.gz',
 '../../../data/ADAM/Dataset006_

In [54]:
results_df = calculate_metrics(num_classes=2, gt_vols_fp=gt_vols_fp, pred_vols_fn=pred_vols_fn)
results_df

100%|██████████| 23/23 [08:17<00:00, 21.64s/it]


,vol_name,dice_aneur_kostas,recall_aneur_kostas,precision_aneur_kostas,dice_tm,recall_tm,precision_tm,mhd
0,10047B.nii.gz,7.051282e-01,5.445545e-01,1.000000e+00,0.683230,0.544554,1.0,0.658391
1,10048B.nii.gz,9.092219e-01,8.335535e-01,1.000000e+00,0.805361,0.833553,1.0,0.334366
2,10029.nii.gz,9.938144e-01,9.877049e-01,1.000000e+00,0.635884,0.987705,1.0,0.652393
3,10024.nii.gz,9.649485e-01,9.322709e-01,1.000000e+00,0.738170,0.932271,1.0,0.429060
4,10068F.nii.gz,9.572854e-01,9.180704e-01,1.000000e+00,0.794829,0.918070,1.0,2.879623
5,10072F.nii.gz,0.000000e+00,0.000000e+00,1.000000e-12,0.000000,0.000000,0.0,inf
6,10037.nii.gz,1.800000e-01,9.890110e-02,1.000000e+00,0.174757,0.098901,1.0,1.836261
7,10068B.nii.gz,9.368132e-01,8.811370e-01,1.000000e+00,0.806147,0.881137,1.0,2.345334
8,10072B.nii.gz,5.132586e-01,3.452239e-01,1.000000e+00,0.512974,0.345224,1.0,8.524691
9,10047F.nii.gz,5.027322e-01,3.357664e-01,1.000000e+00,0.502732,0.335766,1.0,0.929189


Note it could not identify aneurysms at all in 2 cases, scan 61 and scan 1. The rest of them have a very high value except for 51.

In [55]:
os.makedirs(os.path.join(data_dir, 'results'), exist_ok=True)

In [56]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_adam__td_adam__binary_treated_and_untreated_UIAs.csv'), index=False)

In [57]:
results_df.dice_tm.mean()

0.43987249121393845

In [58]:
import numpy as np
results_df.mhd.map(lambda x: x if x != np.inf else 68).mean()


12.599364806442384

### Binary: Untreated UIAs only (predictions not available yet)

In [ ]:
ground_truth_dir = os.path.join(data_dir, 'ADAM/Dataset006_ADAMBinaryAneurysmsOnly')
gt_vols_fp = glob.glob(os.path.join(ground_truth_dir, 'labelsTs', '*.nii.gz'))
predictions_dir = os.path.join(data_dir, 'nnUNet_predictions', 'train_on_SD_predict_on_SD', 'Dataset007_ADAMBinaryUntreatedAneurysmsOnly', 'imageTs')
pred_vols_fn = os.listdir(predictions_dir)
pred_vols_fn[0:5]

In [ ]:
results_df = calculate_metrics(num_classes=2, gt_vols_fp=gt_vols_fp, pred_vols_fn=pred_vols_fn)
results_df

Note it could not identify aneurysms at all in 2 cases, scan 61 and scan 1. The rest of them have a very high value except for 51.

In [ ]:
os.makedirs(os.path.join(data_dir, 'results'), exist_ok=True)

In [ ]:
results_df.to_csv(os.path.join(data_dir, 'results', 'sd_adam__td_adam__binary_only_untreated_UIAs.csv'), index=False)

In [ ]:
results_df.dice_tm.mean()

In [ ]:
import numpy as np
results_df.mhd.map(lambda x: x if x != np.inf else 68).mean()